# Sentence-BERT Inter-Paragraph Coherence

Compute inter-paragraph coherence scores for cw responses using Sentence-BERT embeddings and average cosine similarity.

Run on Google Colab.

In [18]:
# sentence-transformers package setup
!pip install sentence-transformers


In [19]:
# Google Drive setup
from google.colab import drive
drive.mount('/content/drive')
filename_to_load = '/content/drive/My Drive/ANLP23_Project_Data/actual_cw_passage_content.xlsx'
filename_to_save = '/content/drive/My Drive/ANLP23_Project_Data/actual_cw_passage_content_with_inter_paragraph_sim.xlsx'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
# Get passage content

import pandas as pd

# Read the Excel file
df = pd.read_excel(filename_to_load)

# Display the first few rows of the dataframe
print(df.head())


                     model_task_method  conversation_number  \
0  gpt4_cw_ape_zero_shot_cot_responses                    1   
1  gpt4_cw_ape_zero_shot_cot_responses                    2   
2  gpt4_cw_ape_zero_shot_cot_responses                    3   
3  gpt4_cw_ape_zero_shot_cot_responses                    4   
4  gpt4_cw_ape_zero_shot_cot_responses                    5   

                                            response  \
0  Joe was an astronaut with a rather unusual hob...   
1  There lived a hawk on the hill, tall and proud...   
2  Being an outdoors enthusiast, one might consid...   
3  Edward was a corporate executive, laden with p...   
4  Joe was a curious and inventive character, alw...   

                replace_slash_n_slash_n_with_newline  \
0  Joe was an astronaut with a rather unusual hob...   
1  There lived a hawk on the hill, tall and proud...   
2  Being an outdoors enthusiast, one might consid...   
3  Edward was a corporate executive, laden with p...   
4  J

In [21]:
# Packages
from math import sqrt
from sentence_transformers import SentenceTransformer
import numpy as np
import nltk
nltk.download('punkt')
from tqdm import tqdm
tqdm.pandas()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [22]:
# Load model
sentence_model = SentenceTransformer('sentence-transformers/all-distilroberta-v1')


In [23]:
# Testing embedding creation
embedding=sentence_model.encode("this is a sentence")
print(embedding.shape)


(768,)


In [24]:
# Cosine similarity between embeddings
def cosine(one, two):
  return np.dot(one,two)/(sqrt(np.dot(one,one)) * sqrt(np.dot(two,two)))


In [31]:
# Trying to split responses into paragraphs

# Split on newlines
paragraph_lists = [response.split('\n') for response in df['replace_slash_n_with_newline_values']]
mtms = list(df['model_task_method'])
cns = list(df['conversation_number'])

# Print 100 random elements of paragraph_lists
import random
random.seed(42)
print(random.sample(paragraph_lists, 100))


[["John walked up to the fence surrounding the museum and stopped. He was confused. The fence was tall enough to discourage intruders, but there wasn't a locked gate; instead, the bolts were left open. The fence was confused about whether it was supposed to keep things in or keep things out.", 'John considered his options carefully. He hesitated and looked up at the night sky, the clouds illuminated by the nearby lights. He had a choice to make: go in or stay out. Finally, he decided to take a chance. He quietly entered the museum as the super bowl started.'], ["My neighbor Jake was something of a character. He drove a beat-up old truck that, while it may have seen better days, still had a surprising amount of pep to it. Jake's driving was as much a part of his character as his laugh, which was loud and self-assured. Though he was fun to be around, his sense of humor often extended to his driving. It was always dangerous to drive with him since he insisted the safety cones were a slalo

In [32]:
# Statistics of paragraph length
passage_para_lengths = [len(plist) for plist in paragraph_lists]
print('mean length')
print(np.mean(passage_para_lengths))
print('min length')
print(np.min(passage_para_lengths))
print('max length')
print(np.max(passage_para_lengths))
# Print out non-conforming paragraphs
anomaly_counter = 0
for index in range(len(passage_para_lengths)):
  if passage_para_lengths[index] != 2:
    print('anomaly')
    print(paragraph_lists[index])
    print(mtms[index])
    print(cns[index])
    print(passage_para_lengths[index])
    anomaly_counter = anomaly_counter + 1
print('num anomalies')
print(anomaly_counter)


mean length
1.918125
min length
1
max length
4
anomaly
["As a child, Sam was always the fittest kid in the neighborhood. He was different not in his physique, but in his discipline and conviction to learn acrobatics. He would spend countless hours in the backyard, hustling, falling, repeating, and constantly pushing his limits. His quest began with a simple cartwheel and progressed with the complex and seemingly impossible handstands. With consistent practice, perseverance, and a bit of courage, he finally managed to learn it. He found that it isn't difficult to do a handstand if you just stand on your hands. Years later, Sam channelled his physical stamina and mental tenacity towards achieving his lifelong dream - becoming an astronaut. The rigorous training and simulations were reminiscent of his childhood acrobatic drills, yet nothing prepared him for the actual experiences in space. The most startling revelation was the aroma that filled his helmet during his first spacewalk. It ca

In [27]:
# Function to get average cosine similarity between paragraph embeddings - averages of sentence embeddings
def get_passage_interparagraph_cosine_sim(paragraph_list):
    # List of paragraph embeddings
    paragraph_embeddings = []
    # List of paragraph number of sentences
    paragraph_num_sentences = []
    # Loop through paragraphs
    for paragraph in paragraph_list:
        # Split into sentences
        sentences = nltk.sent_tokenize(paragraph)
        # Get sentence embeddings
        sentence_embeddings = [sentence_model.encode(sentence) for sentence in sentences]
        # Average sentence embeddings
        paragraph_embeddings.append(np.mean(sentence_embeddings, axis=0))
        # Saving number of sentences there were embeddings for
        paragraph_num_sentences.append(len(sentence_embeddings))
    # Number of paragraphs
    num_paragraphs = len(paragraph_embeddings)
    # Get cosine similarity between paragraph and the one after it
    cosine_sim = []
    if num_paragraphs > 1:
        for i in range(len(paragraph_embeddings)-1):
            cosine_sim.append(cosine(paragraph_embeddings[i], paragraph_embeddings[i+1]))
        avg_cosine_sim = np.mean(cosine_sim)
    else:
        avg_cosine_sim = 1
    # Return average cosine similarity
    return avg_cosine_sim, num_paragraphs, paragraph_num_sentences


In [28]:
# Test function
test_1_avg, test_1_num_para, test_1_num_sent = get_passage_interparagraph_cosine_sim(["My life has been filled with hardships, but I continue to power forward. I'm a firm believer that every difficulty I face can be used to strengthen my character. I'm not the same person I was at the start, and for that I'm thankful. I'm a living furnace, forging my character so that I may become a stronger, more successful person.", "In some ways, life is like a rose - beautiful, but with thorns. The thorns represent those moments of hardship, challenge, and heartache we face. Yet despite the difficulties we might endure, there is still beauty to be found in life. There's a reason that roses have thorns, despite their beauty. They provide the strength and courage to help us overcome any obstacle we may face."])
print('test 1')
print(test_1_avg)
print(test_1_num_para)
print(test_1_num_sent)

test_2_avg, test_2_num_para, test_2_num_sent = get_passage_interparagraph_cosine_sim(['The boat sailed out of the harbor. Paul joined the other construction workers on the job site.', 'Hi, my name is Bob.', 'Ouch, that hurt.'])
print('test 2')
print(test_2_avg)
print(test_2_num_para)
print(test_2_num_sent)


test 1
0.4835480511380878
2
[4, 5]
test 2
0.1155728341639507
3
[2, 1, 1]


In [29]:
# Create lists of average_cosine_sim, passage_num_paragraphs, passage_num_sentences
average_cosine_sims_responses = []
passage_num_paragraphs_responses = []
passage_num_sentences_responses = []
for plist in tqdm(paragraph_lists):
  avg, passage_num_paragraphs, passage_num_sentences = get_passage_interparagraph_cosine_sim(plist)
  average_cosine_sims_responses.append(avg)
  passage_num_paragraphs_responses.append(passage_num_paragraphs)
  passage_num_sentences_responses.append(passage_num_sentences)


  5%|▌         | 80/1600 [01:48<34:30,  1.36s/it]


KeyboardInterrupt: ignored

In [ ]:
# Create column for average cosine similarity
#df[['avg_cosine_sim', 'num_sentences']] = df.progress_apply(lambda row: get_passage_avg_cosine_sim(row['response']), axis=1)


In [ ]:
# Create dataframe columns
df['avg_inter_paragraph_cosine_sim'] = average_cosine_sims_responses
df['num_paragraphs'] = passage_num_paragraphs_responses
df['num_sentences'] = passage_num_sentences_responses


In [ ]:
# For avg_inter_paragraph_cosine_sim, replace 1 with NaN
df['avg_inter_paragraph_cosine_sim'] = df['avg_inter_paragraph_cosine_sim'].replace(1, np.nan)


In [ ]:
# Write df to Excel
df.to_excel(filename_to_save)
